In [82]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone

from dune_client.client import DuneClient
from config import key

# set pandas to display all rows for inspection 
pd.set_option('display.max_rows', None)

In [74]:
# import json from dune api 
dune = DuneClient(key)
query_result = dune.get_latest_result_dataframe(1442422)

In [75]:
# manually input kiln rewards
kiln_rewards = 161.2943

In [91]:
df_raw = query_result

# drop moving average columns
df = df_raw.drop(columns=['moving_average_eth', 'moving_average_dai', 'running_total_display_curr'])
# rename columns
df.rename(columns={'running_net_eth_display_curr': 'ETH',
                   'running_net_dai_display_curr': 'DAI',
                   'running_net_rpl_display_curr': 'rETH',
                   'running_net_lido_display_curr': 'stETH',
                   'running_net_enzyme_display_curr': 'Enzyme',
                  }, inplace=True)


# turn <nil> values into 0 for rETH and turn into floats
df['rETH'] = df['rETH'].replace('<nil>', '0').astype('float')
# change day to a datetime
df['day'] = pd.to_datetime(df['day'])

# Adjust Enzyme for Kiln returns

    # create datetime object of 2023/08/30 when the Kiln validators were activated 
kiln_deposit_date = datetime(year=2023, month=8, day=30, tzinfo=timezone.utc)

    # work out days since kiln deposit from the latest day and work out the rewards per day
days_since_kiln = (df.iloc[0]['day'] - kiln_deposit_date)/timedelta(days=1)
daily_kiln_rewards = kiln_rewards/days_since_kiln

    # create a column for kiln rewards based on the daily values since validators went live
df['kiln_rewards'] = daily_kiln_rewards * (df['day'] - kiln_deposit_date) / timedelta(days=1)
df.loc[df['kiln_rewards'] < 0, 'kiln_rewards'] = 0

    # Add Kiln rewards to Enzyme column and then drop the Kiln rewards column
df['Enzyme'] = df['Enzyme'] + df['kiln_rewards']
df.drop(columns=['kiln_rewards'])

# add all the columns except day together to get numeric total
df['sum'] = df.drop(columns=['day']).sum(axis=1)

In [92]:
df

,day,ETH,DAI,rETH,stETH,Enzyme,kiln_rewards,sum
0,2024-03-19 00:00:00+00:00,30496.775503,1336.194735,14717.445707,33407.071931,13306.162003,161.294300,93424.944177
1,2024-03-18 00:00:00+00:00,30561.712842,1302.880167,14715.648871,33403.455425,13305.363516,160.495813,93449.556634
2,2024-03-17 00:00:00+00:00,30646.568115,1282.669179,14712.544693,33400.597511,13304.617256,159.697327,93506.694080
3,2024-03-16 00:00:00+00:00,30766.323833,1266.443960,14712.684259,33397.569480,13303.870988,158.898840,93605.791360
4,2024-03-15 00:00:00+00:00,30865.350441,1254.686495,14715.163803,33394.620348,13303.124736,158.100353,93691.046177
5,2024-03-14 00:00:00+00:00,30946.815007,1244.588913,14715.448327,33391.245804,13302.378476,157.301867,93757.778394
6,2024-03-13 00:00:00+00:00,31018.521261,1249.237872,14714.527498,33388.102967,13301.632224,156.503380,93828.525202
7,2024-03-12 00:00:00+00:00,31143.968973,1260.607020,14713.384357,33384.772025,13300.885965,155.704894,93959.323233
8,2024-03-11 00:00:00+00:00,31246.024579,1276.627960,14711.320975,33381.411130,13300.139720,154.906407,94070.430771
9,2024-03-10 00:00:00+00:00,31384.190689,1298.531494,14710.300522,33378.133584,13299.393462,154.107920,94224.657670


Index([202], dtype='int64')